In [1]:
from dotenv import load_dotenv

import altair as alt
import polars as pl

from src.epicorAPI.CSIProducts import get_csi_sales, categories, material_codes, competitors

load_dotenv()

True

In [2]:
def print_price(num: float):
    return f"${num:,.2f}"

In [3]:
csi_products = get_csi_sales()
csi_products

local


changedate,ordernum,partnum,unitprice,linedesc
datetime[μs],i64,str,f64,str
2022-05-25 00:00:00,21472,"""BFL-SGL-TF011""",141.55,"""- Single Baffles TF011 …"
2022-05-25 00:00:00,21472,"""BFL-SGL-TF011""",54.93,"""- Single Baffles TF011 …"
2022-05-25 00:00:00,21472,"""BFL-SGL-TF011""",43.95,"""- Single Baffles TF011 …"
2022-01-26 00:00:00,21490,"""BIO-PSS-SP""",1619.2,"""Plant Screen Sphere - 3' Dia"""
2022-06-07 00:00:00,21663,"""BFL-FLD-001""",300.0,"""- Folded Baffles 001 …"
…,…,…,…,…
2025-08-21 00:00:00,26984,"""CLD-WFD-001-SND-CUST""",71563.36,"""Soundcore, Waffle Folded Cloud…"
2025-08-21 00:00:00,26985,"""BFL-DRP-001-PSH-108X8""",358.61,"""PoshFelt, Draped Baffles 001, …"
2025-08-21 00:00:00,26985,"""SYS-LNR-HRL-3CBL""",137.06,"""Aluminum channel with substrat…"


In [4]:
total_revenue = csi_products['unitprice'].sum()
f"${total_revenue:,.2f}"

'$4,969,534.29'

In [5]:
def get_category_revenue_sum(df: pl.DataFrame, category_id: str):
    return (df.filter(pl.col('partnum').str.contains(category_id)))['unitprice'].sum()


def category_revenue(df: pl.DataFrame, total_rev: float):
    category_revenue_list = []
    for cat in categories:
        rev = get_category_revenue_sum(df, cat)
        # print(f"{category}\t\t${revenue:,.2f}", f"{(revenue / total_revenue) * 100:.2f}%")
        category_revenue_list.append({
            "category": cat,
            "price": f"{rev:,.2f}",
            "percentage": float(f"{(rev / total_rev) * 100:.2f}"),
            "revenue": float(f"{rev:.2f}"),
        })

    return pl.DataFrame(category_revenue_list)


category_revenue_df = category_revenue(csi_products, total_revenue)
print(f"Total revenue: {total_revenue:,.2f}")
category_revenue_df.select(["category", "price", "percentage"]).sort(pl.col("percentage"), descending=True)

Total revenue: 4,969,534.29


category,price,percentage
str,str,f64
"""SUR""","""2,058,924.42""",41.43
"""CLD""","""1,232,769.01""",24.81
"""BFL""","""1,211,322.81""",24.37
"""BIO""","""131,029.16""",2.64
"""DVD""","""106,469.40""",2.14
…,…,…
"""HDW""","""89,768.13""",1.81
"""SYS""","""33,896.93""",0.68
"""LED""","""15,872.98""",0.32


In [6]:
def get_material_revenue(df: pl.DataFrame, material_code: str):
    return (df.filter(pl.col('partnum').str.contains(material_code)))['unitprice'].sum()


def material_revenue(df: pl.DataFrame, total_rev: float) -> pl.DataFrame:
    material_revenue_list = []
    for mat in material_codes:
        rev = get_material_revenue(df, mat)
        material_revenue_list.append({
            "material": mat,
            "price": print_price(rev),
            "percentage": float(f"{(rev / total_rev) * 100:.2f}"),
            "revenue": float(f"{rev:.2f}"),
        })

    return pl.DataFrame(material_revenue_list)


material_revenue_df = material_revenue(csi_products, total_revenue)
print(f"Total revenue: {total_revenue:,.2f}")
material_revenue_df.select(["material", "price", "percentage"]).sort(pl.col("percentage"), descending=True)

Total revenue: 4,969,534.29


material,price,percentage
str,str,f64
"""SND""","""$618,997.52""",12.46
"""SWT""","""$616,766.14""",12.41
"""PSH""","""$292,642.84""",5.89
"""SWT2""","""$2,304.48""",0.05


In [7]:
# comp_sales = pl.read_csv("../data/competitorCategorySales.csv", truncate_ragged_lines=True)
# comp_sales

comp_sales = csi_products.filter(
    pl.any_horizontal(
        pl.col("partnum")
        .str.split_exact("-", 2)
        .struct[2].str.starts_with(competitor) for competitor in list(competitors.keys())
    )
)

In [8]:
comp_total_revenue = comp_sales['unitprice'].sum()
print_price(comp_total_revenue)

'$2,191,914.23'

In [9]:

comp_category_revenue_df = category_revenue(comp_sales, comp_total_revenue)

print(f"Total revenue: {comp_total_revenue:,.2f}")
comp_category_revenue_df.sort(pl.col("percentage"), descending=True)

Total revenue: 2,191,914.23


category,price,percentage,revenue
str,str,f64,f64
"""SUR""","""1,094,661.81""",49.94,1.0947e6
"""BFL""","""719,946.86""",32.85,719946.86
"""CLD""","""205,912.72""",9.39,205912.72
"""GRD""","""80,803.68""",3.69,80803.68
"""DVD""","""47,545.51""",2.17,47545.51
…,…,…,…
"""SYS""","""3,525.47""",0.16,3525.47
"""LED""","""0.00""",0.0,0.0
"""HDW""","""24.92""",0.0,24.92


In [10]:
comp_material_revenue_df = material_revenue(comp_sales, comp_total_revenue)

print(f"Total revenue: {comp_total_revenue:,.2f}")
comp_material_revenue_df.sort(pl.col("percentage"), descending=True)

Total revenue: 2,191,914.23


material,price,percentage,revenue
str,str,f64,f64
"""SND""","""$169,859.73""",7.75,169859.73
"""PSH""","""$115,584.60""",5.27,115584.6
"""SWT""","""$48,356.03""",2.21,48356.03
"""SWT2""","""$124.30""",0.01,124.3


In [11]:
def competitor_summary(df: pl.DataFrame, total_comp_revenue: float):
    competitor_summary_list = []
    for comp in competitors:
        competitor_orders = df.filter(
            pl.any_horizontal(
                pl.col("partnum")
                .str.split_exact("-", 2)
                .struct[2].str.starts_with(comp)
            )
        )
        competitor_summary_list.append({
            "competitor": competitors[comp],
            "competitorId": comp,
            "quoteCount": len(competitor_orders),
            "totalRevenue": float(competitor_orders['unitprice'].sum()),
            "totalRevenuePrice": print_price(float(competitor_orders['unitprice'].sum())),
            "percentage": float(f"{(competitor_orders['unitprice'].sum() / total_comp_revenue) * 100:.2f}"),
        })

    return pl.DataFrame(competitor_summary_list)


comp_summary = (competitor_summary(comp_sales, comp_total_revenue)
                .filter(pl.col("totalRevenue") > 0)
                .sort("percentage", descending=True)
                .select(["competitor", "totalRevenuePrice", "percentage"]))
comp_summary

competitor,totalRevenuePrice,percentage
str,str,f64
"""FilzFelt""","""$1,077,714.81""",49.17
"""Turf""","""$682,105.24""",31.12
"""Sonus""","""$99,734.89""",4.55
"""Zintra""","""$84,307.02""",3.85
"""Acoufelt""","""$57,989.25""",2.65
…,…,…
"""Fact Design""","""$115.96""",0.01
"""Skutchi""","""$142.52""",0.01
"""Soelberg""","""$151.22""",0.01


In [12]:
df = comp_summary.to_pandas()

chart = alt.Chart(df).mark_arc().encode(
    theta=alt.Theta(field="percentage", type="quantitative"),
    color=alt.Color(field="competitor", type="nominal"),
    tooltip=["competitor", "percentage"]
)

chart.to_html("chart.html")

with open("chart.html", "w") as f:
    f.write(chart.to_html())

chart.show()

alt.Chart(...)

In [17]:
comp_category_revenue_df.to_pandas().to_dict(orient="records")

[{'category': 'SUR',
  'price': '1,094,661.81',
  'percentage': 49.94,
  'revenue': 1094661.81},
 {'category': 'LED', 'price': '0.00', 'percentage': 0.0, 'revenue': 0.0},
 {'category': 'DVD',
  'price': '47,545.51',
  'percentage': 2.17,
  'revenue': 47545.51},
 {'category': 'HDW', 'price': '24.92', 'percentage': 0.0, 'revenue': 24.92},
 {'category': 'BIO',
  'price': '42,544.96',
  'percentage': 1.94,
  'revenue': 42544.96},
 {'category': 'LBR', 'price': '0.00', 'percentage': 0.0, 'revenue': 0.0},
 {'category': 'BFL',
  'price': '719,946.86',
  'percentage': 32.85,
  'revenue': 719946.86},
 {'category': 'CLD',
  'price': '205,912.72',
  'percentage': 9.39,
  'revenue': 205912.72},
 {'category': 'SYS',
  'price': '3,525.47',
  'percentage': 0.16,
  'revenue': 3525.47},
 {'category': 'DSGNSER', 'price': '0.00', 'percentage': 0.0, 'revenue': 0.0},
 {'category': 'GRD',
  'price': '80,803.68',
  'percentage': 3.69,
  'revenue': 80803.68}]

In [19]:
comp_category_revenue_df.to_dicts()

[{'category': 'SUR',
  'price': '1,094,661.81',
  'percentage': 49.94,
  'revenue': 1094661.81},
 {'category': 'LED', 'price': '0.00', 'percentage': 0.0, 'revenue': 0.0},
 {'category': 'DVD',
  'price': '47,545.51',
  'percentage': 2.17,
  'revenue': 47545.51},
 {'category': 'HDW', 'price': '24.92', 'percentage': 0.0, 'revenue': 24.92},
 {'category': 'BIO',
  'price': '42,544.96',
  'percentage': 1.94,
  'revenue': 42544.96},
 {'category': 'LBR', 'price': '0.00', 'percentage': 0.0, 'revenue': 0.0},
 {'category': 'BFL',
  'price': '719,946.86',
  'percentage': 32.85,
  'revenue': 719946.86},
 {'category': 'CLD',
  'price': '205,912.72',
  'percentage': 9.39,
  'revenue': 205912.72},
 {'category': 'SYS',
  'price': '3,525.47',
  'percentage': 0.16,
  'revenue': 3525.47},
 {'category': 'DSGNSER', 'price': '0.00', 'percentage': 0.0, 'revenue': 0.0},
 {'category': 'GRD',
  'price': '80,803.68',
  'percentage': 3.69,
  'revenue': 80803.68}]

In [21]:

start_date = "2025-07-01"
end_date = "2025-07-30"
csi_sales = get_csi_sales(startdate=start_date, enddate=end_date)
csi_sales

local


changedate,ordernum,partnum,unitprice,linedesc
datetime[μs],i64,str,f64,str
2025-07-25 00:00:00,24696,"""SUR-PFL-TF028""",76.02,"""- Soundcore Profile Surfaces T…"
2025-07-25 00:00:00,26039,"""BFL-DRP-001-PSH-108X7""",283.07,"""PoshFelt, Draped Baffles 001, …"
2025-07-25 00:00:00,26039,"""BFL-DRP-001-PSH-108X9""",308.87,"""PoshFelt, Draped Baffles 001, …"
2025-07-25 00:00:00,26039,"""BFL-DRP-001-PSH-108X10""",343.67,"""PoshFelt, Draped Baffles 001, …"
2025-07-25 00:00:00,26039,"""BFL-DRP-001-PSH-108X12""",378.47,"""PoshFelt, Draped Baffles 001, …"
…,…,…,…,…
2025-07-29 00:00:00,26877,"""BFL-SGL-AR003-SND-36X12""",20.54,"""Soundcore, Single Baffles AR00…"
2025-07-29 00:00:00,26877,"""BFL-SGL-AR003-SND-27X12""",15.39,"""Soundcore, Single Baffles AR00…"
2025-07-29 00:00:00,26877,"""SYS-LNR-HRL-3CBL""",191.4,"""Aluminum channel with substrat…"


In [22]:
from src.epicorAPI.CSIProductAnalysis import get_total_revenue

cat_asdf = category_revenue(csi_sales, get_total_revenue(csi_sales))
cat_asdf

category,price,percentage,revenue
str,str,f64,f64
"""SUR""","""45,740.00""",34.19,45740.0
"""LED""","""2,352.00""",1.76,2352.0
"""DVD""","""3,432.96""",2.57,3432.96
"""HDW""","""11,211.99""",8.38,11211.99
"""BIO""","""0.00""",0.0,0.0
…,…,…,…
"""BFL""","""6,546.30""",4.89,6546.3
"""CLD""","""55,323.82""",41.36,55323.82
"""SYS""","""3,941.66""",2.95,3941.66


In [23]:
comp_summary = (competitor_summary(comp_sales, comp_total_revenue)
                .filter(pl.col("totalRevenue") > 0)
                .sort("percentage", descending=True))
comp_summary

competitor,competitorId,quoteCount,totalRevenue,totalRevenuePrice,percentage
str,str,i64,f64,str,f64
"""FilzFelt""","""FF""",106,1.0777e6,"""$1,077,714.81""",49.17
"""Turf""","""TF""",217,682105.24,"""$682,105.24""",31.12
"""Sonus""","""SN""",12,99734.89,"""$99,734.89""",4.55
"""Zintra""","""ZI""",59,84307.02,"""$84,307.02""",3.85
"""Acoufelt""","""AF""",56,57989.25,"""$57,989.25""",2.65
…,…,…,…,…,…
"""Fact Design""","""FT""",1,115.96,"""$115.96""",0.01
"""Skutchi""","""SK""",1,142.52,"""$142.52""",0.01
"""Soelberg""","""SB""",1,151.22,"""$151.22""",0.01


In [24]:
comp_summary.to_dicts()

[{'competitor': 'FilzFelt',
  'competitorId': 'FF',
  'quoteCount': 106,
  'totalRevenue': 1077714.8099999998,
  'totalRevenuePrice': '$1,077,714.81',
  'percentage': 49.17},
 {'competitor': 'Turf',
  'competitorId': 'TF',
  'quoteCount': 217,
  'totalRevenue': 682105.24,
  'totalRevenuePrice': '$682,105.24',
  'percentage': 31.12},
 {'competitor': 'Sonus',
  'competitorId': 'SN',
  'quoteCount': 12,
  'totalRevenue': 99734.88999999998,
  'totalRevenuePrice': '$99,734.89',
  'percentage': 4.55},
 {'competitor': 'Zintra',
  'competitorId': 'ZI',
  'quoteCount': 59,
  'totalRevenue': 84307.02,
  'totalRevenuePrice': '$84,307.02',
  'percentage': 3.85},
 {'competitor': 'Acoufelt',
  'competitorId': 'AF',
  'quoteCount': 56,
  'totalRevenue': 57989.25,
  'totalRevenuePrice': '$57,989.25',
  'percentage': 2.65},
 {'competitor': 'Armstrong',
  'competitorId': 'AR',
  'quoteCount': 30,
  'totalRevenue': 41630.54,
  'totalRevenuePrice': '$41,630.54',
  'percentage': 1.9},
 {'competitor': 'FSor